In [ ]:
import ee
import geemap

def sample_point_merge(img, aoi):
    # Sample each image at a point and bring results to pandas
    # Note: geemap.ee_to_df expects an ee.FeatureCollection. Mapping a function
    # over an ImageCollection that returns Features still yields an ImageCollection,
    # so we explicitly wrap the mapped results with ee.FeatureCollection(...).

    # Compute mean over the AOI for each image. Use bestEffort to avoid failures
    # on very large geometries.
    mean = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=10,
        bestEffort=True,
    )
    # Return a Feature with the band's mean values and the image date.
    return ee.Feature(aoi, mean).set('date', img.date().format('YYYY-MM-dd'))

ee.Initialize()

s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR")
)
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

# Filter both by date and region
region = ee.Geometry.Point([-120.5, 37.5])
start = '2023-05-01'
end = '2023-08-01'

s2_filtered = s2.filterBounds(region).filterDate(start, end)
l8_filtered = l8.filterBounds(region).filterDate(start, end)

# Merge the two collections
combined = s2_filtered.merge(l8_filtered)
combined = ee.FeatureCollection(s2.map(lambda img: sample_point_merge(img, region)))

# df = geemap.ee_to_df(combined)
# print("Combined ImageCollection:", df)


Combined ImageCollection: ee.Number({
  "functionInvocationValue": {
    "functionName": "Collection.size",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Element.set",
                    "arguments": {
                      "key": {
                        "constantValue": "date"
                      },
                      "object": {
                        "functionInvocationValue": {
                          "functionName": "Feature",
                          "arguments": {
                            "geometry": {
                              "functionInvocationValue": {
                                "fu